# SMIB system as in Milano's book example 8.1

In [1]:
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pydae.ssa as ssa

## Import system module

In [3]:
from smib_milano_ex8p1 import smib_milano_ex8p1_class

## Instantiate system

In [4]:
syst = smib_milano_ex8p1_class()
syst.t_end = 15.0
syst.Dt = 0.01
syst.decimation =1
syst.update()

## Simulate

In [5]:
events = [{'t_end':1.0,  'P_t':0.0,'Q_t':0.0},  # initialize with P_t = 0.0 and Q_t = 0.0 and run 1 s
          {'t_end':10.0, 'p_m':0.5,'e1q':1.2,'D':20}]  # change p_m to 0.5 and e1q to 1.2 and until 10 s


syst.simulate(events,xy0=1);

### Benchmark using numba

In [6]:
%timeit syst.simulate(events,xy0=1)

9.61 ms ± 445 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Results plots

In [7]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(6, 4), sharex = True)
axes[0].plot(syst.T, syst.get_values('delta'))
axes[1].plot(syst.T, syst.get_values('omega'))
axes[2].plot(syst.T, syst.get_values('P_t'))

y_labels = ['$\delta$','$\omega$','$P_t$']
for ax,y_label in zip(axes,y_labels):
    ax.set_ylabel(y_label)
    ax.set_xlim((0,10))
    ax.grid(True)
ax.set_xlabel('Time (s)')    


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Time (s)')

## Small signal analysis

In [22]:
syst.initialize(events,xy0=1);
A = ssa.eval_A(syst)

In [23]:
ssa.damp_report(syst)

,Real,Imag,Freq.,Damp
Mode 1,-1.428571,11.984147,1.907336,0.118367
Mode 2,-1.428571,-11.984147,1.907336,0.118367


In [24]:
ssa.participation(syst).abs().round(2)

,Mode 1,Mode 2
delta,0.5,0.5
omega,0.5,0.5
